In [1]:
!ls data/han/AWS/

남이섬_2016.xlsx  남이섬_2020.xlsx  청평_2019.xlsx  화천_2018.xlsx
남이섬_2017.xlsx  청평_2016.xlsx    청평_2020.xlsx  화천_2019.xlsx
남이섬_2018.xlsx  청평_2017.xlsx    화천_2016.xlsx  화천_2020.xlsx
남이섬_2019.xlsx  청평_2018.xlsx    화천_2017.xlsx


In [2]:
!pwd

/home/cjinw/work/water-quality/rnn


In [3]:
%matplotlib widget
import pandas as pd
import numpy as np
from glob import glob
import os
import datetime
import matplotlib.pyplot as plt
    
from tensorflow.keras.layers import Input, Concatenate, Dot, Add, ReLU, Activation
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow import keras

In [4]:
# for Korean in plot
import matplotlib
import matplotlib.font_manager as fm
fm.get_fontconfig_fonts()
font_location = '/usr/share/fonts/truetype/nanum/NanumGothicCoding.ttf'
#font_location = '/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc'
# font_location = 'C:/Windows/Fonts/NanumGothic.ttf' # For Windows
fprop = fm.FontProperties(fname=font_location)

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:

folder = 'data/han/자동'
# folder = 'data/han/수질'
# folder = 'data/han/AWS'
# folder = 'data/nakdong/자동'

## 한강
## 자동
file_names = [['가평_2016.xlsx','가평_2017.xlsx','가평_2018.xlsx', '가평_2019.xlsx'], 
              ['의암호_2016.xlsx','의암호_2017.xlsx','의암호_2018.xlsx', '의암호_2019.xlsx'],
              ['서상_2016.xlsx','서상_2017.xlsx','서상_2018.xlsx', '서상_2019.xlsx']]

## 수질
# file_names = [['가평천3_2016.xlsx','가평천3_2017.xlsx','가평천3_2018.xlsx', '가평천3_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천댐1_2016.xlsx','춘천댐1_2017.xlsx','춘천댐1_2018.xlsx', '춘천댐1_2019.xlsx'],
#               ['대성리_2016.xlsx','대성리_2017.xlsx','대성리_2018.xlsx', '대성리_2019.xlsx'],
#               ['의암_2016.xlsx','의암_2017.xlsx','의암_2018.xlsx', '의암_2019.xlsx'],
#               ['춘성교_2016.xlsx','춘성교_2017.xlsx','춘성교_2018.xlsx', '춘성교_2019.xlsx'],
#               ['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2019.xlsx'],
#               ['청평댐3_2016.xlsx','청평댐3_2017.xlsx','청평댐3_2018.xlsx', '청평댐3_2019.xlsx'],
#               ['춘천_2016.xlsx','춘천_2017.xlsx','춘천_2018.xlsx', '춘천_2019.xlsx']
#              ]


## AWS
# file_names = [['남이섬_2016.xlsx','남이섬_2017.xlsx','남이섬_2018.xlsx', '남이섬_2018.xlsx'],
#               ['청평_2016.xlsx', '청평_2017.xlsx', '청평_2018.xlsx', '청평_2019.xlsx'],
#               ['화천_2016.xlsx', '화천_2017.xlsx', '화천_2018.xlsx', '화천_2019.xlsx']]
 

    
## 낙동강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]


## 금강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
             

    
    
## 영산강
#자동
# file_names = [['도개_2016.xlsx','도개_2017.xlsx','도개_2018.xlsx', '도개_2019.xlsx'],
#               ['신암_2016.xlsx','신암_2017.xlsx','신암_2018.xlsx', '신암_2019.xlsx'],
#               ['회상_2016.xlsx','회상_2017.xlsx','회상_2018.xlsx', '회상_2019.xlsx']]
    


In [7]:
target_col = '용존산소'
# 각각의 강 마다, 측정소 별로 column을 맞춰야함
# 0: 자동, 1: 수질, 2: AWS(ASOS), 
han = [':,2:11', ':,28:45', ':,26:29']
nakdong = [':,2:11', ':,2:12']
# gain_han = ['save/han/1', 'save/han/2', 'save/han/3']

# interpolation_option = True
interpolation_option = False
# gain_save_path = gain_han[2]

iloc_val = han[0]
iloc_val

':,2:11'

In [8]:
def make_timeseries(df, interpolate=None):
    print(df.shape)
    date_col = df.columns[0]
    df[date_col] = pd.to_datetime(df[date_col])
    df = df[df[date_col].notna()]

    year = pd.DatetimeIndex(df[date_col]).year.astype(np.int64)

    start = str(year[0]) + "-01-01 00:00"
#     start

    end = str(year[0]) + "-12-31 23:00"
#     end

    time_series = pd.date_range(start = start, end = end, freq='H')
    time_series
    time_series = pd.DataFrame(time_series)
    time_series.columns = [date_col]   
    time_series

    df = pd.concat([time_series, df], axis=0)
    df = df.drop_duplicates([date_col], keep="last")
    df = df.sort_values([date_col], axis=0)
    
    if interpolate :
        print('interpolation')
        df = df.interpolate(metod='spline', order=2, limit_direction='both')
        
    print(df.shape)
    return df

In [9]:
def make_dataframe(file_names, iloc_val, interpolate=None):
    day = 24*60*60
    year = (365.2425)*day

    df_full = []
    df = []
    
    for loc in range(len(file_names)):
    
        df_loc = []
        for y in range(len(file_names[loc])):
            path = os.path.join(folder, file_names[loc][y])
            print(file_names[loc][y])
            df_tmp = make_timeseries(pd.read_excel(path), interpolate=interpolate)
            
    #         df_tmp = pd.read_excel(path).dropna(axis='columns', how = 'all')
    #         print(df_tmp.head) 
            df_loc.append(df_tmp)
#             df_loc.append(pd.read_excel(path))
        df_full.append(pd.concat(df_loc))
    #     df.append(df_full[loc].iloc[:,2:11])
        df.append(eval(f"df_full[loc].iloc[{iloc_val}]"))
    #     df.append(df_full[loc].iloc[:, [2,3,4,5,6,7,10]])
    #     df.append(df_full[loc].iloc[:, 2:11])
        date_time = pd.to_datetime(df_full[loc].iloc[:, 0], format='%Y.%m.%d %H:%M', utc=True)
        timestamp_s = date_time.map(datetime.datetime.timestamp)
        df[loc]['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
        df[loc]['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
        df[loc]['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
        df[loc]['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
        df[loc] = df[loc].reset_index(drop=True)
        
#         print(df[loc].shape[0])
#         rows = df[loc].shape[0]
#         percentage = round(rows * 0.7)
#         df[loc] = df[loc].dropna(axis='columns',  thresh=percentage)
        

    
    return df

In [10]:
def normalize(df):
    # normalize data
    
    
    df_all = pd.concat(df)
    
    train_mean = df_all.mean()
    train_std = df_all.std()
    for i in range(len(file_names)):
        df[i] = (df[i]-train_mean)/train_std
    
    return df_all, train_mean, train_std

In [11]:
##for normal
# df = make_dataframe(file_names, iloc_val)
## for interpolation
df = make_dataframe(file_names, iloc_val, interpolate=interpolation_option)
df_all, train_mean, train_std = normalize(df)

가평_2016.xlsx
(8784, 12)
(8784, 12)
가평_2017.xlsx
(8744, 12)
(8760, 12)
가평_2018.xlsx
(8730, 12)
(8760, 12)
가평_2019.xlsx
(8760, 12)
(8760, 12)
의암호_2016.xlsx
(8784, 11)
(8784, 11)
의암호_2017.xlsx
(8760, 11)
(8760, 11)
의암호_2018.xlsx
(8746, 11)
(8760, 11)
의암호_2019.xlsx
(8760, 11)
(8760, 11)
서상_2016.xlsx
(8784, 11)
(8784, 11)
서상_2017.xlsx
(8760, 11)
(8760, 11)
서상_2018.xlsx
(8751, 11)
(8760, 11)
서상_2019.xlsx
(8760, 11)
(8760, 11)


In [12]:
train_mean, train_std

(수온          1.371826e+01
 수소이온농도      7.363691e+00
 전기전도도       1.062274e+02
 용존산소        1.048823e+01
 탁도          2.190726e+00
 총유기탄소       1.599534e+00
 총질소         1.617681e+00
 총인          8.346780e-03
 클로로필-a      8.673920e+00
 Day sin     6.037103e-15
 Day cos     2.747682e-15
 Year sin   -5.681266e-08
 Year cos    2.053380e-05
 dtype: float64,
 수온           7.365852
 수소이온농도       0.482657
 전기전도도       13.561701
 용존산소         2.071821
 탁도           8.264139
 총유기탄소        0.357371
 총질소          0.398078
 총인           0.007764
 클로로필-a       9.531307
 Day sin      0.707110
 Day cos      0.707110
 Year sin     0.707103
 Year cos     0.707117
 dtype: float64)

In [13]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.265062,0.903974,-0.016769,1.067547,-0.192485,-0.838161,NaN,NaN,-0.616276,-1.999586e-12,1.414207,-0.003771,1.414158
1,-1.292214,0.903974,-0.016769,1.067547,-0.192485,-0.838161,NaN,NaN,-0.626768,3.660237e-01,1.366019,-0.002757,1.414161
2,-1.332943,0.903974,0.056968,0.971013,-0.192485,-0.838161,NaN,NaN,-0.626768,7.071034e-01,1.224739,-0.001743,1.414162
3,-1.373671,0.696788,0.056968,0.971013,-0.192485,-0.838161,NaN,NaN,-0.616276,9.999952e-01,0.999995,-0.000730,1.414163
4,-1.400824,0.696788,-0.016769,0.971013,-0.192485,-0.838161,NaN,NaN,-0.637260,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.319367,0.075228,0.351917,0.826213,-0.216686,0.001303,-0.084608,0.084135,0.527323,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-1.387248,0.075228,0.425654,0.874480,-0.216686,0.001303,-0.298133,0.084135,0.464373,-1.224739e+00,0.707103,-0.007096,1.414145
35061,-1.441552,0.075228,0.425654,0.922746,-0.216686,0.281124,-0.072047,0.084135,0.422406,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-1.482281,0.075228,0.499391,0.971013,-0.216686,0.001303,-0.325766,0.212936,0.411914,-7.071034e-01,1.224739,-0.005068,1.414154


In [14]:
df[2]

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,-1.265062,0.903974,-0.016769,1.067547,-0.192485,-0.838161,NaN,NaN,-0.616276,-1.999586e-12,1.414207,-0.003771,1.414158
1,-1.292214,0.903974,-0.016769,1.067547,-0.192485,-0.838161,NaN,NaN,-0.626768,3.660237e-01,1.366019,-0.002757,1.414161
2,-1.332943,0.903974,0.056968,0.971013,-0.192485,-0.838161,NaN,NaN,-0.626768,7.071034e-01,1.224739,-0.001743,1.414162
3,-1.373671,0.696788,0.056968,0.971013,-0.192485,-0.838161,NaN,NaN,-0.616276,9.999952e-01,0.999995,-0.000730,1.414163
4,-1.400824,0.696788,-0.016769,0.971013,-0.192485,-0.838161,NaN,NaN,-0.637260,1.224739e+00,0.707103,0.000284,1.414163
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,-1.319367,0.075228,0.351917,0.826213,-0.216686,0.001303,-0.084608,0.084135,0.527323,-1.366019e+00,0.366024,-0.008109,1.414140
35060,-1.387248,0.075228,0.425654,0.874480,-0.216686,0.001303,-0.298133,0.084135,0.464373,-1.224739e+00,0.707103,-0.007096,1.414145
35061,-1.441552,0.075228,0.425654,0.922746,-0.216686,0.281124,-0.072047,0.084135,0.422406,-9.999952e-01,0.999995,-0.006082,1.414150
35062,-1.482281,0.075228,0.499391,0.971013,-0.216686,0.001303,-0.325766,0.212936,0.411914,-7.071034e-01,1.224739,-0.005068,1.414154


In [15]:
# just for Create WidnowGenerator 
train_df = df[0]
val_df = df[0]
test_df = df[0]

In [16]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
            #train_df=None, val_df=None, test_df=None,
#                out_features = None,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [17]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [18]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [19]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result
@property
def example2(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.val))
    # And cache it for next time
    self._example = result
  return result
@property
def example3(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result


WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example
WindowGenerator.example2 = example2
WindowGenerator.example3 = example3

In [20]:
def sample_batch_index(total, batch_size):
    '''Sample index of the mini-batch.

    Args:
        - total: total number of samples
        - batch_size: batch size

    Returns:
        - batch_idx: batch index
    '''
    total_idx = np.random.permutation(total)
    batch_idx = total_idx[:batch_size]
    return batch_idx

In [21]:
def binary_sampler(p, shape):
  '''Sample binary random variables.
  
  Args:
    - p: probability of 1
    - shape: matrix shape
    
  Returns:
    - binary_random_matrix: generated binary random matrix.
  '''
  unif_random_matrix = np.random.uniform(0., 1., size = shape)
  binary_random_matrix = 1*(unif_random_matrix < p)
  return binary_random_matrix

In [22]:
def uniform_sampler(low, high, shape):
  '''Sample uniform random variables.
  
  Args:
    - low: low limit
    - high: high limit
    - rows: the number of rows
    - cols: the number of columns
    
  Returns:
    - uniform_random_matrix: generated uniform random matrix.
  '''
  return np.random.uniform(low, high, size = shape)

In [23]:
class MissData(object):
    def __init__(self, load_dir=None):
        if load_dir:
            self.missarr = np.load(os.path.join(load_dir, 'miss.npy'))
            self.idxarr = np.load(os.path.join(load_dir, 'idx.npy'))
            
    def make_missdata(self, data_x, missrate=0.2):
        data = data_x.copy()
        rows, cols = data_x.shape
        total_no = rows*cols
        total_miss_no = np.round(total_no*missrate).astype(int)
        total_idx = self.idxarr.shape[0]
        idxarr = self.idxarr
        missarr = self.missarr
        #print(total_miss_no)
        miss_no = 0
        cum_no = self.idxarr[:,3:4]
        cum_no = cum_no.reshape((total_idx))
        cum_sum = np.max(cum_no)
        #print(cum_no)
        #print(total_idx)
        while True:
            loc_count = np.around(np.random.random()*cum_sum)
            #print('loc_count =', loc_count)
            idx = len(cum_no[cum_no <= loc_count])-1
            #print(cum_no[cum_no <= loc_count])
            #print('idx =', idx)
            startnan = idxarr[idx][0]
            nanlen = idxarr[idx][2]
            loc = np.around(np.random.random()*(rows-nanlen)).astype(int)
            #print('loc =', loc)
            #print(loc_count, idx)
            #print(idxarr[idx])
            #data_copy = data[loc:loc+nanlen].copy()
            data_copy = data[loc:loc+nanlen]
            #print('startnan=', startnan)
            #isnan = missarr[startnan:startnan+nanlen].copy()
            isnan = missarr[startnan:startnan+nanlen]
            #print('isnan =',isnan)
            miss_no += idxarr[idx][1]
            if (miss_no > total_miss_no):
                break
            data_copy[isnan==1] = np.nan
            data[loc:loc+nanlen] = data_copy
        #print('miss_data =', data)
        return data
    
    def save(data, max_tseq, save_dir='save'):
        no, dim = data.shape
        #print((no, dim))
        isnan = np.isnan(data).astype(int)
        isany = np.any(isnan, axis=1).astype(int)
        shifted = np.roll(isany, 1)
        shifted[0] = 1
        #print(isnan)
        #print(isany.astype(int))
        #print(shifted)
        startnan = ((isany == 1) & (shifted ==0)).astype(int)
        #print(startnan)
        group = startnan.cumsum()
        group = group*isany
#         print(group)
        n = np.max(group)
#         print(n)
        missarr = None
        cum_no = 0
        rowidx = 0
        for i in range(1, n+1):
            g = (group == i).astype(int)
            i = np.argmax(g)
            rows = g.sum()
            #print(len)
            #print(i)
            #print(type(missarr))
            if rows <= max_tseq:
                nanseq = isnan[i:i+rows, :]
                no = np.sum(nanseq)
                #print(no)
                if missarr is None:
                    missarr = nanseq
                    idxarr = np.array([[rowidx, no, rows, cum_no]])
                else:
                    missarr = np.concatenate((missarr, nanseq))
                    idxarr = np.concatenate((idxarr, [[rowidx, no, rows, cum_no]]), axis=0)
                cum_no += no
                rowidx += rows

        print(idxarr)
        miss_npy_file = os.path.join(save_dir, 'miss.npy')
        idx_npy_file = os.path.join(save_dir, 'idx.npy')
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        np.save(miss_npy_file, missarr)
        np.save(idx_npy_file, idxarr)
        print('miss_data file saved')

In [24]:
def interpolate(np_data, max_gap=3):
    #n = np_data.shape[1]
    data = pd.DataFrame(np_data)
    #data[0][0] = np.nan
    #data[0][1] = np.nan
    #data[0][2] = np.nan
    #data[data.columns[0]][0] = np.nan
    #data[data.columns[0]][1] = np.nan
    #data[data.columns[0]][2] = np.nan
    
    # create mask
    mask = data.copy()
    grp = ((mask.notnull() != mask.shift().notnull()).cumsum())
    grp['ones'] = 1
    for i in data.columns:
        mask[i] = (grp.groupby(i)['ones'].transform('count') < max_gap) | data[i].notnull()
    data = data.interpolate(method='polynomial', order=5, limit=max_gap, axis=0).bfill()[mask]
    return data.to_numpy()
    #return data
    
#filled_data = interpolate(norm_data, max_gap=3)
#np.arange(0, 5, dtype=int)
#['%d'%val for val in range(0,5)]

In [25]:
class GainDataGenerator(keras.utils.Sequence):
    'Generates data for GAIN'
    def __init__(self,
                 data_list,
                 batch_size=32,
                 input_width=24*3,
                 label_width=24*3,
                 shift=0,
                 fill_no=4,
                 miss_rate=0.2,
                 hint_rate=0.9,
                 normalize=True,
                 miss_pattern=None,
                 miss_data_load_dir='save',
                 alpha=100.):
        'Initialization'
        window_size = input_width
        
        # interpollation
        filled_data = []
        for data in data_list:
            data = interpolate(data, max_gap=fill_no)
            filled_data.append(data)
            
        data_list = filled_data
        
        # whole data
        self.data = np.concatenate(data_list)

        # TO-DO
        
        # pre calculation for  sequence data
        last_cum = 0
        cums = []
        for data in data_list:
            isnan = np.isnan(data)
            isany = np.any(isnan, axis=1)
            #shift same as pd.shift(isany, fill_value=True)
            shifted = np.roll(isany, 1)
            shifted[0] = True # set to nan
            
            start_seq = ((isany == False) & (shifted == True)).astype(int)
            cum = start_seq.cumsum()
            cum += last_cum
            last_cum = np.max(cum)
            cum[isany] = 0
            cums.append(cum)

        # Define mask matrix
        if miss_pattern is None:
            print("pattern none")
            self.data_m = binary_sampler(1-miss_rate, self.data.shape)
        else:
            #MissData.save(self.data, max_tseq = 12)
            print("load save")
            self.miss = MissData(load_dir=miss_data_load_dir)
            self.miss_rate = miss_rate
            miss_data = self.miss.make_missdata(self.data, self.miss_rate)
            self.data_m = 1. - np.isnan(miss_data).astype(float)
            
            self.data_m_rand = binary_sampler(1-(miss_rate/10.), self.data.shape)
            self.data_m[self.data_m_rand==0.] = 0.
        self.miss_pattern = miss_pattern
        
        # sequence data
        self.ids = np.concatenate(cums)
        data_idx = np.empty((0), dtype=int)
        for i in range(1, last_cum+1):
            seq_len = (self.ids == i).sum()
            start_id = np.argmax(self.ids == i)
            # possible data number in seqeunce
            time_len = seq_len - window_size + 1
            start_ids = np.arange(start_id, start_id+time_len)
            data_idx = np.append(data_idx, start_ids)
            
        # start index set for sequence data
        self.data_idx = data_idx
        self.input_width = input_width
        self.no = len(data_idx)
        
        #print('self.no = ', self.no)
        
        self.batch_size = batch_size
        
        # random shuffling  index
        self.batch_idx = sample_batch_index(self.no, self.no)
        self.batch_id = 0
        self.shape = (batch_size,self.input_width)+self.data.shape[1:]
        #self.hint_rate = hint_rate
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        # Sample batch
        x = np.empty((0, self.input_width, self.data.shape[1]))
        #m = np.empty((0, self.input_width, self.data.shape[1]))
        #h = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.input_width, self.data.shape[1]))
        #print(x.shape)
        #print(self.data.shape)
        #print(self.input_width)
        #self.batch_idx = sample_batch_index(self.no, self.batch_size)
        for cnt in range(0, self.batch_size):
            i = self.batch_idx[self.batch_id]
            self.batch_id += 1
            #self.batch_id %= self.batch_size
            self.batch_id %= self.no
            if self.miss_pattern and (self.batch_id == 0):
                self.batch_idx = sample_batch_index(self.no, self.no)
                miss_data = self.miss.make_missdata(self.data, self.miss_rate)
                self.data_m = 1. - np.isnan(miss_data).astype(float)
                self.data_m_rand = binary_sampler(1-self.miss_rate/10., self.data.shape)
                self.data_m[self.data_m_rand==0.] = 0.
            idx1 = self.data_idx[i]
            idx2 = self.data_idx[i]+self.input_width
            #print(idx1, idx2)
        
            Y_mb = self.data[idx1:idx2].copy()
            X_mb = Y_mb.copy()
            M_mb = self.data_m[idx1:idx2]
            Z_mb = uniform_sampler(0, 0.01, shape=X_mb.shape)
            X_mb = M_mb*X_mb + (1-M_mb)*Z_mb
            #H_mb_temp = binary_sampler(self.hint_rate, shape=X_mb.shape)
            #H_mb = M_mb * H_mb_temp
            X_mb[M_mb == 0] = np.nan
            Y_mb[M_mb == 1] = np.nan
            x = np.append(x, [X_mb], axis=0)
            #m = np.append(m, [M_mb], axis=0)
            #h = np.append(h, [H_mb], axis=0)
            y = np.append(y, [Y_mb], axis=0)
            
        #return [x, m, h], y
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

# dgen = GainDataGenerator(df)

In [26]:
class GAIN(keras.Model):
    def __init__(self, shape, alpha=100., load=False, hint_rate=0.9, gen_sigmoid=True, **kwargs):
        super(GAIN, self).__init__(**kwargs)
        self.shape = shape
        self.dim = np.prod(shape).astype(int)
        self.h_dim = self.dim
        self.gen_sigmoid = gen_sigmoid
        self.build_generator()
        self.build_discriminator()
        self.hint_rate = hint_rate
        self.alpha = alpha
        self.generator_optimizer = Adam()
        self.discriminator_optimizer = Adam()

    ## GAIN models
    def build_generator(self):
        last_activation = 'sigmoid' if self.gen_sigmoid else None
        xavier_initializer = tf.keras.initializers.GlorotNormal()

        shape = self.shape
        #x = Input(shape=(self.dim,), name='generator_input_x')
        #m = Input(shape=(self.dim,), name='generator_input_m')
        x = Input(shape=shape, name='generator_input_x')
        m = Input(shape=shape, name='generator_input_m')
        
        x_f = Flatten()(x)
        m_f = Flatten()(m)

        a = Concatenate()([x_f, m_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        #a = keras.layers.BatchNormalization()(a)
        a = Dense(self.dim, activation=last_activation, kernel_initializer=xavier_initializer)(a)
        G_prob = keras.layers.Reshape(shape)(a)
        self.generator = keras.models.Model([x, m], G_prob, name='generator')

    def build_discriminator(self):
        xavier_initializer = tf.keras.initializers.GlorotNormal()
        shape = self.shape

        #x = Input(shape=(self.dim,), name='discriminator_input_x')
        #h = Input(shape=(self.dim,), name='discriminator_input_h')
        x = Input(shape=shape, name='discriminator_input_x')
        h = Input(shape=shape, name='discriminator_input_h')
        
        x_f = Flatten()(x)
        h_f = Flatten()(h)

        a = Concatenate()([x_f, h_f])

        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.h_dim, activation='relu', kernel_initializer=xavier_initializer)(a)
        a = Dense(self.dim, activation='sigmoid', kernel_initializer=xavier_initializer)(a)
        D_prob = keras.layers.Reshape(shape)(a)
        self.discriminator = keras.models.Model([x, h], D_prob, name='discriminator')
        
    def call(self, inputs):
        if isinstance(inputs, tuple):
            inputs = inputs[0]
        shape = inputs.shape
        dims = np.prod(shape[1:])
        input_width = shape[1]
        #print('inputs.shape=',inputs.shape)
        x = inputs
        #x = x.reshape((n, -1))
        #print('dims=',dims)
        #x = keras.layers.Reshape((dims,))(x)
        #x = keras.layers.Reshape(tf.TensorShape((self.dim,)))(x)
        #print('x =', x)
        #print('x.shape = ', x.shape)
        #x = keras.layers.Reshape(tf.TensorShape([57]))(x)
        
        isnan = tf.math.is_nan(x)
        #m = 1.- keras.backend.cast(isnan, dtype=tf.float32)
        m = tf.where(isnan, 0., 1.)
        z = keras.backend.random_uniform(shape=tf.shape(x), minval=0.0, maxval=0.01)
        x = tf.where(isnan, z, x)
        #z = uniform_sampler(0, 0.01, shape=x.shape)
        #z = tf.keras.backend.random_uniform(shape=x.shape, minval=0.0, maxval=0.01)
        imputed_data = self.generator([x, m], training=False)
        #imputed_data = m*x + (1-m)*imputed_data
        #imputed_data = tf.where(isnan, imputed_data, np.nan)
        imputed_data = tf.where(isnan, imputed_data, x)
        #imputed_data = keras.layers.Reshape(shape[1:])(imputed_data)
        #print('imputed_data.shape = ', imputed_data.shape)
        
        return imputed_data
    
    def D_loss(M, D_prob):
        ## GAIN loss
        return -tf.reduce_mean(M * tf.keras.backend.log(D_prob + 1e-8) \
                         + (1-M) * tf.keras.backend.log(1. - D_prob + 1e-8))
    
    def G_loss(self, M, D_prob, X, G_sample):
        G_loss_temp = -tf.reduce_mean((1-M) * tf.keras.backend.log(D_prob + 1e-8))
        MSE_loss = tf.reduce_mean((M * X - M * G_sample)**2) / (tf.reduce_mean(M) + 1e-8)
        #G_loss_temp = GAIN.G_loss_bincross(M, D_prob)
        #MSE_loss = GAIN.MSE_loss(M, X, G_sample)
        G_loss = G_loss_temp + self.alpha * MSE_loss
        return G_loss
        
    def RMSE_loss(y_true, y_pred):
        isnan = tf.math.is_nan(y_true)
        M = tf.where(isnan, 1., 0.)
        return tf.sqrt(tf.reduce_sum(tf.where(isnan, 0., y_pred-y_true)**2)/tf.reduce_sum(1-M))
    
    def train_step(self, data):
        #[x, m, h], y = data
        x, y = data
        #X = keras.layers.Reshape((self.dim,), input_shape=self.shape)(x)
        #Y = keras.layers.Reshape((self.dim,), input_shape=self.shape)(y)
        #X = keras.layers.Flatten()(x)
        #Y = keras.layers.Flatten()(y)
        X = x
        Y = y
        #X = tf.reshape(x, shape=(x.shape[0], -1))
        #Y = tf.reshape(y, shape=(x.shape[0], -1))
        isnan = tf.math.is_nan(X)
        #M = 1 - keras.backend.cast(isnan, dtype=tf.float32)
        M = tf.where(isnan, 0., 1.)
        Z = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=0.01)
        #H_temp = binary_sampler(self.hint_rate, shape=X.shape)
        H_rand = keras.backend.random_uniform(shape=tf.shape(X), minval=0.0, maxval=1.)
        #H_temp = 1*keras.backend.cast((H_rand < self.hint_rate), dtype=tf.float32)
        H_temp = tf.where(H_rand < self.hint_rate, 1., 0.)
        
        H = M * H_temp
        #X = M * X + (1-M) * Z
        X = tf.where(isnan, Z, X)
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            G_sample = self.generator([X, M], training=True)

            # Combine with observed data
            #Hat_X = tf.where(isnan, G_sample, X)
            Hat_X = X * M + G_sample * (1-M)
            D_prob = self.discriminator([Hat_X, H], training=True)
            gen_loss = self.G_loss(M, D_prob, X, G_sample)
            disc_loss = tf.keras.backend.mean(tf.keras.losses.binary_crossentropy(M, D_prob))
            #disc_loss = GAIN.D_loss(M, D_prob)
            #disc_loss = GAIN.D_loss(M, D_prob)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.generator_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))
        
        rmse = tf.sqrt(tf.reduce_sum(tf.where(isnan, G_sample - Y, 0.)**2)/tf.reduce_sum(1-M))
        return {
                 'gen_loss':     gen_loss,
                 'disc_loss':    disc_loss,
                 'rmse':         rmse,
               }
    
    def save(self, save_dir='save'):
        if not os.path.exists(save_dir):
          os.makedirs(save_dir)
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        self.discriminator.save_weights(disc_savefile)
        self.generator.save_weights(gen_savefile)

    def load(self, save_dir='save'):
        disc_savefile = os.path.join(save_dir, 'discriminator.h5')
        gen_savefile = os.path.join(save_dir, 'generator.h5')
        try:
          self.discriminator.load_weights(disc_savefile)
          self.generator.load_weights(gen_savefile)
          print('model weights loaded')
        except:
          print('model loadinng error')

In [27]:
def make_dataset_gain(self, data):
  dg = GainDataGenerator(
      df,
      input_width = self.input_width,
      label_width = self.label_width,
      batch_size = 128,
      normalize = False,
      miss_pattern = True,
      miss_rate = 0.15,
      fill_no = 3,
  )
  self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.shape,
        dg.shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_gain

# 측정망별 반복 구간 

In [28]:
norm_df = pd.concat(df,axis=0)
norm_data = norm_df.to_numpy()
MissData.save(norm_data, max_tseq = 24)
# 1: 자동, 2: 수질, 

[[    0    22     3     0]
 [    3     9     1    22]
 [    4    14     2    31]
 ...
 [ 5451     9     9 19454]
 [ 5460     3     3 19463]
 [ 5463     7     1 19466]]
miss_data file saved


In [29]:
dgen = GainDataGenerator(df)

pattern none


In [30]:
train_df = df_all
val_df = df_all
test_df = df_all
df_all

,수온,수소이온농도,전기전도도,용존산소,탁도,총유기탄소,총질소,총인,클로로필-a,Day sin,Day cos,Year sin,Year cos
0,4.5,7.5,126.0,10.8,0.6,1.4,2.398,0.006,5.7,-1.407890e-12,1.000000,-0.002666,0.999996
1,4.3,7.5,126.0,10.8,0.5,1.4,2.490,0.003,5.4,2.588190e-01,0.965926,-0.001950,0.999998
2,4.4,7.5,126.0,10.8,0.5,1.4,2.545,0.003,5.4,5.000000e-01,0.866025,-0.001233,0.999999
3,4.4,7.5,126.0,10.8,0.6,1.4,2.482,0.002,5.4,7.071068e-01,0.707107,-0.000516,1.000000
4,4.4,7.5,126.0,10.8,0.6,1.4,2.453,0.003,5.4,8.660254e-01,0.500000,0.000201,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,4.0,7.4,111.0,12.2,0.4,1.6,1.584,0.009,13.7,-9.659258e-01,0.258819,-0.005734,0.999984
35060,3.5,7.4,112.0,12.3,0.4,1.6,1.499,0.009,13.1,-8.660254e-01,0.500000,-0.005017,0.999987
35061,3.1,7.4,112.0,12.4,0.4,1.7,1.589,0.009,12.7,-7.071068e-01,0.707107,-0.004301,0.999991
35062,2.8,7.4,113.0,12.5,0.4,1.6,1.488,0.010,12.6,-5.000000e-01,0.866025,-0.003584,0.999994


In [31]:
wide_window = WindowGenerator(
    input_width=24*5, label_width=24*5, shift=0,
    #label_columns=['T (degC)']
)

wide_window

Total window size: 120
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]
Label indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 

## Graph 

In [32]:
wide_window.example[0].shape

load save


TensorShape([128, 120, 13])

In [33]:
wide_window.plot(plot_col=target_col)
print('make_dataset_gain: dg.no = ', wide_window.dg.no)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

make_dataset_gain: dg.no =  9335


In [34]:
plt.figure(figsize=(9,10))
isnan = np.isnan(norm_data).astype(int)
data = isnan
n = data.shape[0]
seq_len = n//8
for i in range(8):
    plt.subplot(181+i)
    plt.imshow(data[i*seq_len:(i+1)*seq_len, 0:], aspect='auto')
    plt.yticks([])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 컴파일 및 학습_GAIN

In [35]:
val_performance = {}
performance = {}

In [36]:
gain = GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False)
gain.compile(loss=GAIN.RMSE_loss)

In [37]:
MAX_EPOCHS = 2000

def compile_and_fit(model, window, patience=10):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  #model.compile(loss=tf.losses.MeanSquaredError(),
                #optimizer=tf.optimizers.Adam(),
                #metrics=[tf.metrics.MeanAbsoluteError()])
  model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

**모델 학습**

In [38]:
history = compile_and_fit(gain, wide_window, patience=MAX_EPOCHS//5)


val_performance['Gain'] = gain.evaluate(wide_window.val)
performance['Gain'] = gain.evaluate(wide_window.test, verbose=0)

# gain.save(save_dir='save')

load save
load save
Epoch 1/2000
1/1 [==============================] - 1s 1s/step - gen_loss: 127.8562 - disc_loss: 0.7306 - rmse: 1.0933 - val_loss: 0.8668
Epoch 2/2000
1/1 [==============================] - 0s 102ms/step - gen_loss: 99.7281 - disc_loss: 0.5228 - rmse: 0.9914 - val_loss: 0.8709
Epoch 3/2000
1/1 [==============================] - 0s 146ms/step - gen_loss: 78.4187 - disc_loss: 0.5115 - rmse: 0.8890 - val_loss: 0.8024
Epoch 4/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 65.4093 - disc_loss: 0.4189 - rmse: 0.7123 - val_loss: 0.7516
Epoch 5/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 57.1844 - disc_loss: 0.3915 - rmse: 0.7498 - val_loss: 0.7513
Epoch 6/2000
1/1 [==============================] - 0s 122ms/step - gen_loss: 56.8527 - disc_loss: 0.4159 - rmse: 0.7054 - val_loss: 0.7020
Epoch 7/2000
1/1 [==============================] - 0s 145ms/step - gen_loss: 43.2410 - disc_loss: 0.3940 - rmse: 0.7000 - val_loss: 0.6500
Ep

1/1 [==============================] - 0s 119ms/step - gen_loss: 6.4123 - disc_loss: 0.3037 - rmse: 0.3121 - val_loss: 0.3045
Epoch 60/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 5.5444 - disc_loss: 0.3095 - rmse: 0.2815 - val_loss: 0.3233
Epoch 61/2000
1/1 [==============================] - 0s 152ms/step - gen_loss: 7.5482 - disc_loss: 0.3029 - rmse: 0.2788 - val_loss: 0.3185
Epoch 62/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 6.1237 - disc_loss: 0.3003 - rmse: 0.2941 - val_loss: 0.3272
Epoch 63/2000
1/1 [==============================] - 0s 103ms/step - gen_loss: 5.4005 - disc_loss: 0.2905 - rmse: 0.2908 - val_loss: 0.3175
Epoch 64/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 7.3106 - disc_loss: 0.2868 - rmse: 0.2712 - val_loss: 0.3028
Epoch 65/2000
1/1 [==============================] - 0s 160ms/step - gen_loss: 5.2925 - disc_loss: 0.2907 - rmse: 0.2945 - val_loss: 0.3250
Epoch 66/2000
1/1 [===============

1/1 [==============================] - 0s 118ms/step - gen_loss: 4.8611 - disc_loss: 0.2439 - rmse: 0.2482 - val_loss: 0.2563
Epoch 118/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 4.5586 - disc_loss: 0.2436 - rmse: 0.2673 - val_loss: 0.2815
Epoch 119/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 4.9985 - disc_loss: 0.2380 - rmse: 0.2727 - val_loss: 0.2630
Epoch 120/2000
1/1 [==============================] - 0s 129ms/step - gen_loss: 5.2496 - disc_loss: 0.2407 - rmse: 0.2720 - val_loss: 0.2874
Epoch 121/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 4.4469 - disc_loss: 0.2434 - rmse: 0.2392 - val_loss: 0.2914
Epoch 122/2000
1/1 [==============================] - 0s 104ms/step - gen_loss: 4.6079 - disc_loss: 0.2397 - rmse: 0.2616 - val_loss: 0.2986
Epoch 123/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 6.4803 - disc_loss: 0.2410 - rmse: 0.2754 - val_loss: 0.3233
Epoch 124/2000
1/1 [========

1/1 [==============================] - 0s 129ms/step - gen_loss: 5.4809 - disc_loss: 0.2311 - rmse: 0.2442 - val_loss: 0.2464
Epoch 176/2000
1/1 [==============================] - 0s 120ms/step - gen_loss: 4.4804 - disc_loss: 0.2143 - rmse: 0.2469 - val_loss: 0.2809
Epoch 177/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 5.7713 - disc_loss: 0.2144 - rmse: 0.2584 - val_loss: 0.2610
Epoch 178/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 3.9694 - disc_loss: 0.2124 - rmse: 0.2364 - val_loss: 0.2391
Epoch 179/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 4.6969 - disc_loss: 0.2107 - rmse: 0.2832 - val_loss: 0.2682
Epoch 180/2000
1/1 [==============================] - 0s 134ms/step - gen_loss: 5.1343 - disc_loss: 0.2130 - rmse: 0.2392 - val_loss: 0.2562
Epoch 181/2000
1/1 [==============================] - 0s 136ms/step - gen_loss: 4.0279 - disc_loss: 0.2112 - rmse: 0.3056 - val_loss: 0.2502
Epoch 182/2000
1/1 [========

1/1 [==============================] - 0s 158ms/step - gen_loss: 5.5338 - disc_loss: 0.2055 - rmse: 0.2789 - val_loss: 0.2351
Epoch 234/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 3.9314 - disc_loss: 0.2100 - rmse: 0.2573 - val_loss: 0.2249
Epoch 235/2000
1/1 [==============================] - 0s 148ms/step - gen_loss: 5.9519 - disc_loss: 0.2003 - rmse: 0.2829 - val_loss: 0.2358
Epoch 236/2000
1/1 [==============================] - 0s 143ms/step - gen_loss: 3.9805 - disc_loss: 0.2033 - rmse: 0.2587 - val_loss: 0.2427
Epoch 237/2000
1/1 [==============================] - 0s 173ms/step - gen_loss: 5.7637 - disc_loss: 0.2022 - rmse: 0.2702 - val_loss: 0.2463
Epoch 238/2000
1/1 [==============================] - 0s 151ms/step - gen_loss: 5.0518 - disc_loss: 0.1983 - rmse: 0.2661 - val_loss: 0.2632
Epoch 239/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 4.2111 - disc_loss: 0.1994 - rmse: 0.2678 - val_loss: 0.2380
Epoch 240/2000
1/1 [========

...

1/1 [==============================] - 0s 147ms/step - gen_loss: 2.6583 - disc_loss: 0.1503 - rmse: 0.2020 - val_loss: 0.2108
Epoch 582/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.5562 - disc_loss: 0.1467 - rmse: 0.3841 - val_loss: 0.2328
Epoch 583/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 2.5778 - disc_loss: 0.1491 - rmse: 0.3925 - val_loss: 0.2249
Epoch 584/2000
1/1 [==============================] - 0s 394ms/step - gen_loss: 3.1050 - disc_loss: 0.1516 - rmse: 0.2251 - val_loss: 0.2458
Epoch 585/2000
1/1 [==============================] - 0s 130ms/step - gen_loss: 4.4492 - disc_loss: 0.1595 - rmse: 0.2273 - val_loss: 0.2353
Epoch 586/2000
1/1 [==============================] - 0s 101ms/step - gen_loss: 3.6214 - disc_loss: 0.1534 - rmse: 0.2172 - val_loss: 0.2398
Epoch 587/2000
1/1 [==============================] - 0s 125ms/step - gen_loss: 3.4493 - disc_loss: 0.1566 - rmse: 0.2433 - val_loss: 0.2411
Epoch 588/2000
1/1 [========

1/1 [==============================] - 0s 128ms/step - gen_loss: 4.0300 - disc_loss: 0.1467 - rmse: 0.2269 - val_loss: 0.2236
Epoch 640/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 4.1809 - disc_loss: 0.1500 - rmse: 0.2307 - val_loss: 0.2184
Epoch 641/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 3.0410 - disc_loss: 0.1498 - rmse: 0.2524 - val_loss: 0.2353
Epoch 642/2000
1/1 [==============================] - 0s 124ms/step - gen_loss: 2.7363 - disc_loss: 0.1500 - rmse: 0.2052 - val_loss: 0.2357
Epoch 643/2000
1/1 [==============================] - 0s 126ms/step - gen_loss: 2.8622 - disc_loss: 0.1531 - rmse: 0.2262 - val_loss: 0.2147
Epoch 644/2000
1/1 [==============================] - 0s 156ms/step - gen_loss: 2.7897 - disc_loss: 0.1431 - rmse: 0.2246 - val_loss: 0.2164
Epoch 645/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.7273 - disc_loss: 0.1542 - rmse: 0.2114 - val_loss: 0.2452
Epoch 646/2000
1/1 [========

1/1 [==============================] - 0s 137ms/step - gen_loss: 2.6161 - disc_loss: 0.1403 - rmse: 0.2174 - val_loss: 0.2387
Epoch 698/2000
1/1 [==============================] - 0s 117ms/step - gen_loss: 2.7066 - disc_loss: 0.1393 - rmse: 0.4122 - val_loss: 0.2385
Epoch 699/2000
1/1 [==============================] - 0s 133ms/step - gen_loss: 2.7255 - disc_loss: 0.1422 - rmse: 0.2178 - val_loss: 0.2284
Epoch 700/2000
1/1 [==============================] - 0s 135ms/step - gen_loss: 2.5541 - disc_loss: 0.1409 - rmse: 0.4051 - val_loss: 0.2390
Epoch 701/2000
1/1 [==============================] - 0s 119ms/step - gen_loss: 2.4797 - disc_loss: 0.1446 - rmse: 0.2192 - val_loss: 0.2406
Epoch 702/2000
1/1 [==============================] - 0s 137ms/step - gen_loss: 2.3276 - disc_loss: 0.1434 - rmse: 0.2593 - val_loss: 0.2355
Epoch 703/2000
1/1 [==============================] - 0s 149ms/step - gen_loss: 2.4721 - disc_loss: 0.1463 - rmse: 0.4065 - val_loss: 0.2445
Epoch 704/2000
1/1 [========

1/1 [==============================] - 0s 127ms/step - gen_loss: 2.5351 - disc_loss: 0.1468 - rmse: 0.2117 - val_loss: 0.2765
Epoch 756/2000
1/1 [==============================] - 0s 144ms/step - gen_loss: 2.7207 - disc_loss: 0.1385 - rmse: 0.2045 - val_loss: 0.2651
Epoch 757/2000
1/1 [==============================] - 0s 118ms/step - gen_loss: 2.4681 - disc_loss: 0.1473 - rmse: 0.2088 - val_loss: 0.2899
Epoch 758/2000
1/1 [==============================] - 0s 141ms/step - gen_loss: 2.3871 - disc_loss: 0.1417 - rmse: 0.2013 - val_loss: 0.2670
Epoch 759/2000
1/1 [==============================] - 0s 158ms/step - gen_loss: 2.5654 - disc_loss: 0.1408 - rmse: 0.2175 - val_loss: 0.3128
Epoch 760/2000
1/1 [==============================] - 0s 140ms/step - gen_loss: 2.5547 - disc_loss: 0.1448 - rmse: 0.1998 - val_loss: 0.2858
Epoch 761/2000
1/1 [==============================] - 0s 161ms/step - gen_loss: 3.8932 - disc_loss: 0.1458 - rmse: 0.2320 - val_loss: 0.2446
Epoch 762/2000
1/1 [========

1/1 [==============================] - 0s 114ms/step - gen_loss: 2.9371 - disc_loss: 0.1410 - rmse: 0.2413 - val_loss: 0.2365
Epoch 814/2000
1/1 [==============================] - 0s 127ms/step - gen_loss: 3.4696 - disc_loss: 0.1425 - rmse: 0.2092 - val_loss: 0.2617
Epoch 815/2000
1/1 [==============================] - 0s 139ms/step - gen_loss: 2.7516 - disc_loss: 0.1339 - rmse: 0.2267 - val_loss: 0.2425
Epoch 816/2000
1/1 [==============================] - 0s 123ms/step - gen_loss: 2.5753 - disc_loss: 0.1433 - rmse: 0.2139 - val_loss: 0.2131
Epoch 817/2000
1/1 [==============================] - 0s 142ms/step - gen_loss: 2.6358 - disc_loss: 0.1391 - rmse: 0.2287 - val_loss: 0.2367
Epoch 818/2000
1/1 [==============================] - 0s 138ms/step - gen_loss: 2.7119 - disc_loss: 0.1387 - rmse: 0.2475 - val_loss: 0.2293
Epoch 819/2000
1/1 [==============================] - 0s 106ms/step - gen_loss: 3.8520 - disc_loss: 0.1401 - rmse: 0.2566 - val_loss: 0.2689
Epoch 820/2000
1/1 [========

**학습 loss history 출력**

In [39]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax.plot(history.history['gen_loss'], label='gen_loss')
ax.plot(history.history['disc_loss'], label='disc_loss')
ax2.plot(history.history['rmse'], label='rmse', color='green')
ax2.plot(history.history['val_loss'], label='val_loss', color='red')
#plt.legend(history.history.keys(), loc='upper right')
#ax.legend(loc='upper center')
ax.legend(loc='upper center')
ax2.legend(loc='upper right')
ax.set_xlabel("epochs")
ax.set_ylabel("loss")
ax2.set_ylabel("rmse")
plt.show()    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**성능 측정**

In [40]:
gain.evaluate(wide_window.test.repeat(), steps=100)

load save
100/100 [==============================] - 3s 33ms/step - loss: 0.2665


0.2665463089942932

**model save**

In [41]:
gain.save(save_dir='save')

**샘플 prediction 출력**

In [42]:
wide_window.plot(gain, plot_col='총유기탄소')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
norm_df = pd.concat(df,axis=0)

In [44]:
data = norm_df.to_numpy()

total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print(unit_shape)
#dim = np.prod(wide_window.dg.shape[1:]).astype(int)
dim = wide_window.dg.shape[1]
print(dim)
n = (total_n//dim)*dim

x = data[0:n].copy()
y_true = data[0:n].copy()

#x = interpolate(x, max_gap=3)

print('x.shape =', x.shape)
x_reshape = x.reshape((-1,)+unit_shape)
print('x_reshape.shape =', x_reshape.shape)
isnan = np.isnan(x_reshape)
print(isnan.sum())
print('y_true.shape=', y_true.shape)
isnan = np.isnan(y_true)
print(isnan.sum())

x_remain = data[-wide_window.dg.shape[1]:].copy()
x_remain_reshape = x_remain.reshape((-1,)+unit_shape)
x_remain_reshape.shape

105192
(120, 13)
120
x.shape = (105120, 13)
x_reshape.shape = (876, 120, 13)
159120
y_true.shape= (105120, 13)
159120


(1, 120, 13)

In [45]:
gain.evaluate(x_reshape, y_true.reshape((-1,)+unit_shape))

28/28 [==============================] - 0s 2ms/step - loss: 0.0000e+00


0.0

In [46]:
y_pred = gain.predict(x_reshape)
y_remain_pred = gain.predict(x_remain_reshape)

In [47]:
y_pred = y_pred.reshape(y_true.shape)
y_remain_pred = y_remain_pred.reshape(x_remain.shape)
print(y_pred.shape, y_remain_pred.shape)
y_pred = np.append(y_pred, y_remain_pred[-(total_n-n):], axis=0)
print(y_pred.shape)

(105120, 13) (120, 13)
(105192, 13)


In [48]:
y_pred[~np.isnan(data)] = np.nan

In [49]:
n = 8
plt.figure(figsize=(9,20))
for i in range(n):
    #plt.subplot('%d1%d'%(n,i))
    plt.subplot(811+i)
    plt.plot(x[:, i])
    plt.plot(y_pred[:, i])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**Imputed Data**

In [50]:
total_n = wide_window.dg.data.shape[0]
print(total_n)
unit_shape = wide_window.dg.shape[1:]
print('unit_shape=', unit_shape)
time_seq = unit_shape[0]
print(time_seq)
n = (total_n//time_seq)*time_seq
print('n=', n)

gans = []
oris = []
for i in range(len(df)):
    x = df[i].to_numpy()
    total_n = x.shape[0]
    n = (total_n//time_seq)*time_seq
    x = x[0:n]
    x_block = x.reshape((-1,)+unit_shape)
    y = gain.predict(x_block)
    y_gan = y.reshape(x.shape)
    
    # cut off sin, cos data
    if (i > 0):
        x = x[:, :-4]
        y_gan = y_gan[:, :-4]
    gans.append(y_gan)
    oris.append(x)
print(x.shape)
print(y_gan.shape)

105192
unit_shape= (120, 13)
120
n= 105120
(35040, 9)
(35040, 9)


# 예측

In [51]:
df[0].head

<bound method NDFrame.head of              수온    수소이온농도     전기전도도      용존산소        탁도     총유기탄소       총질소  \
0     -1.251486  0.282415  1.457973  0.150479 -0.192485 -0.558340  1.960215   
1     -1.278638  0.282415  1.457973  0.150479 -0.204586 -0.558340  2.191326   
2     -1.265062  0.282415  1.457973  0.150479 -0.204586 -0.558340  2.329489   
3     -1.265062  0.282415  1.457973  0.150479 -0.192485 -0.558340  2.171229   
4     -1.265062  0.282415  1.457973  0.150479 -0.192485 -0.558340  2.098379   
...         ...       ...       ...       ...       ...       ...       ...   
35059 -1.373671 -0.546331  0.646865  1.019280 -0.131983  0.560945  0.252009   
35060 -1.387248 -0.546331  0.646865  1.019280 -0.131983  0.281124  0.272106   
35061 -1.400824 -0.546331  0.646865  1.019280 -0.131983  0.281124  0.302251   
35062 -1.400824 -0.546331  0.646865  1.019280 -0.131983  0.281124  0.264570   
35063 -1.414400 -0.546331  0.646865  1.019280 -0.144084  0.281124  0.219352   

             총인    클로

In [109]:
ori = np.concatenate(oris, axis=1)
gan = np.concatenate(gans, axis=1)
print(oris[0].shape, gans[0].shape)
print(oris[1].shape, gans[1].shape)
print(ori.shape, gan.shape)

total_no = ori.shape[0]
train_no = int(total_no*0.7)

train_slice = slice(0, train_no)
val_slice = slice(train_no, None)
test_slice = slice(0, None)

train_df = pd.DataFrame(gan[train_slice])
val_df = pd.DataFrame(gan[val_slice])
test_df = pd.DataFrame(gan[test_slice])

train_ori_df = pd.DataFrame(ori[train_slice])
val_ori_df = pd.DataFrame(ori[val_slice])
test_ori_df = pd.DataFrame(ori[test_slice])

num_features = train_df.shape[1]
#out_num_features = oris[0].shape[1]-4
# out_features = [target_col_idx]
# out_num_features = len(out_features)
# out_num_features

(35040, 13) (35040, 13)
(35040, 9) (35040, 9)
(35040, 31) (35040, 31)


In [53]:
df[0].columns

Index(['수온', '수소이온농도', '전기전도도', '용존산소', '탁도', '총유기탄소', '총질소', '총인', '클로로필-a',
       'Day sin', 'Day cos', 'Year sin', 'Year cos'],
      dtype='object')

In [110]:
MAX_EPOCHS = 150
target_col = '총질소'
num_features = df[0].shape[1]
label_columns_indices = {name: i for i, name in enumerate(df[0])}
target_col_idx = label_columns_indices[target_col]
# target_col_idx
out_features = [target_col_idx]
# out_features = [3,5,6,7,8]
out_num_features = len(out_features)
out_num_features

1

In [111]:
class WaterDataGenerator(keras.utils.Sequence):
    'Generates data for water'
    def __init__(self,
                 imputed_data,
                 ori_data = None,
                 batch_size=32,
                 input_width=24*7,
                 label_width=24*3,
                 shift=24*3,
                 skip_time = None,
                 shuffle = True,
                 out_features = None,
                 out_num_features = None,
                ):
        'Initialization'
        self.window_size = input_width+shift
        self.total_no = imputed_data.shape[0]
        self.data = imputed_data
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.input_shape = (batch_size, input_width, self.data.shape[1])
        self.out_num_features = out_num_features
#         print("out_features")
#         print(out_features)
        if out_features:
            self.out_features = out_features
        else:
            self.out_features = [i for i in range(out_num_features)]
        self.label_shape = (batch_size, label_width, self.out_num_features)
        if (skip_time):
            # TO-DO
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
        else:
            self.no = self.total_no - self.window_size
            self.data_idx = np.arange(0, self.no)
            
        if shuffle:
            self.batch_idx = np.random.permutation(self.no)
        else:
            self.batch_idx = np.arange(0, self.no)
        self.batch_id = 0
        
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        #return int(128/self.batch_size)
        #return 2
        return 1

    def __getitem__(self, index):
        'Generate one batch of data'
        #print('index =', index)
        #print('self.no =', self.no)
        #print('self.total_no =', self.total_no)
        #print('self.batch_id =', self.batch_id)
        # Sample batch
        label_width = self.label_width
        batch_idx = self.batch_idx
        
        x = np.empty((0, self.input_width, self.data.shape[1]))
        
        #y = np.empty((0, self.input_width, self.data.shape[1]))
        y = np.empty((0, self.label_width, self.out_num_features))
       
        #print(x.shape)
        #print(y.shape)
        
        
        for cnt in range(0, self.batch_size):
            i = self.batch_id
            self.batch_id += 1
            idx1 = self.data_idx[batch_idx[i]]
            idx2 = idx1 + self.input_width
            
            X = self.data[idx1:idx2].to_numpy()
            
            
            idx1 = self.data_idx[batch_idx[i]] + self.window_size - label_width
            idx2 = idx1 + label_width
            
            #Y = self.data[idx1:idx2,:,:out_num_features]
            Y = self.data.iloc[idx1:idx2, self.out_features].to_numpy()
            #Y = self.data[idx1:idx2]
            #print('Y.shape = ', Y.shape)
            #Y = Y.iloc[:,:out_num_features]
            
            self.batch_id %= self.no
            #print("x.shape=", x.shape)
            #print('X.shape=', X.shape)
            #print(type(x), type(X))
            x = np.append(x, [X], axis = 0)
            
            y = np.append(y, [Y], axis = 0)
            
        return x, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        return

**WaterDataGenerator 테스트**

In [112]:
# OUT_STEPS = 24*3
# wdg = WaterDataGenerator(train_df,
#                          batch_size=128,
#                          input_width = 24*7,
#                          label_width = OUT_STEPS,
#                          shift = OUT_STEPS,
#                          out_num_features = out_num_features,
# #                          out_features = out_features
#                         )

In [113]:
# train_df

In [114]:
# it = iter(wdg)

In [115]:
# x,y = next(it)
# x.shape, y.shape

In [116]:
def make_dataset_water(self, data):
#   print("make_dataset_water")
#   print(out_features)
  dg = WaterDataGenerator(
      data,
      batch_size = 128,
      input_width = self.input_width,
      label_width = self.label_width,
      shift = self.label_width,
      out_features = out_features,
      out_num_features = out_num_features,
  )
  #self.dg = dg
  ds = tf.data.Dataset.from_generator(
      lambda: dg,
      output_types=(tf.float32, tf.float32),
      output_shapes=(
        dg.input_shape,
        dg.label_shape
        #[batch_size, train_generator.dim],
        #[batch_size, train_generator.dim],
      )
  )
  return ds

WindowGenerator.make_dataset = make_dataset_water

In [117]:
def plot2(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]', fontproperties=fprop)
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
      label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
      label_col_index = plot_col_index
      label_out_col_index = plot_out_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_out_col_index],
                label='Labels', c='#2ca02c')
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_out_col_index],
                  marker=None, label='Predictions',
                  c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot2 = plot2

**1일단위로 변환**

In [118]:
def hour_to_day_mean(array):
    
    time = 24
    return tf.reduce_mean(tf.reshape(array, [array.shape[0]//time, time]), 1)

def plot24(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0):
    
  inputs, labels = self.example
  plt.figure(figsize=(10, 8))
  plot_col_index = self.column_indices[plot_col]
  plot_out_col_index = self.column_indices[plot_out_col]
  max_n = min(max_subplots, len(inputs))

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
    label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
  else:
    label_col_index = plot_col_index
    label_out_col_index = plot_out_col_index
    
  for n in range(max_n):
    plt.subplot(3, 1, n+1)
    plt.ylabel(f'{df_all.columns[plot_col]} [normed]', fontproperties=fprop)
    
    input_temp = hour_to_day_mean(inputs[n, :, plot_col_index])
    input_temp = input_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.input_indices),
        input_temp,
        label='Inputs', marker='.', zorder=-10)

    if label_col_index is None:
      continue
    
    label_temp = hour_to_day_mean(labels[n, :, label_out_col_index])
    label_temp = label_temp * train_std[plot_col] + train_mean[plot_col]
    
    plt.plot(
        hour_to_day_mean(self.label_indices),
        label_temp,
        label='Labels', marker='.', zorder=-10, c='#2ca02c')
    
    if model is not None:
      predictions = model(inputs)
    
      #predictions = predictions * train_std[plot_col] * train_mean[plot_col]
      predict_temp = hour_to_day_mean(predictions[n,:, label_out_col_index])
      predict_temp = predict_temp * train_std[plot_col] + train_mean[plot_col]
    
      plt.plot(
          hour_to_day_mean(self.label_indices),
          predict_temp,
          label='Predictions', marker='.', zorder=-10, c='#ff7f0e')

    if n == 0:
      plt.legend()

  plt.xlabel('Time [day]')

WindowGenerator.plot24 = plot24

**예측률 계산**

In [119]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [120]:
def compa(self, model=None, plot_col=0, max_subplots=3, plot_out_col=0, windows=None, min_max_normailze=False):
       
    if example is not None :
        inputs, labels = windows
    else :
        inputs, labels = self.example
        
        
    plot_out_col_index = self.column_indices[plot_out_col]
    
    if self.label_columns:
        label_out_col_index = self.label_columns_indices.get(plot_out_col, None)
    else:
        label_out_col_index = plot_out_col_index
    
    if model is None:
        return
    
    print("column : " + df_all.columns[plot_col])
    
    mae = 0
    mse = 0
    rmse = 0
    mape = 0
    pred_arr = []
    label_arr = []
    
    predictions = model(inputs)
    
    if min_max_normailze :
        predictions = predictions * train_std[plot_col] + train_mean[plot_col]
        labels = labels * train_std[plot_col] + train_mean[plot_col]
        
        predictions = predictions.numpy()
        labels = labels.numpy()
        
        predictions = (predictions - predictions.min()) / (predictions.max() - predictions.min())
        labels = (labels - labels.min()) / (labels.max() - labels.min())

    for n in range(len(inputs)):
        pred_temp = hour_to_day_mean(predictions[n,:,label_out_col_index]) 
        label_temp = hour_to_day_mean(labels[n,:,label_out_col_index])
        
        pred_arr.append(pred_temp[2])
        label_arr.append(label_temp[2])
        
        
    mae = mean_absolute_error(label_arr, pred_arr)
    mse = mean_squared_error(label_arr, pred_arr)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((np.array(label_arr) - np.array(pred_arr))/np.array(label_arr)) )*100
    
    print("mae:")  
    print(mae)
    
    print("mse:")  
    print(mse)
    
    print("rmse")  
    print(rmse)
    
    print("mape")  
    print(mape)
    
    

WindowGenerator.compa = compa

In [121]:
train_df.shape

(24528, 31)

In [122]:
#multi_window.compa(elman_model, plot_col=out_features[0])

In [123]:
# target_col = '용존산소'
# num_features = df[0].shape[1]
# label_columns_indices = {name: i for i, name in enumerate(df[0])}
# target_col_idx = label_columns_indices[target_col]
# # target_col_idx
# out_features = [target_col_idx]
# # out_features = [3,5,6,7,8]
# out_num_features = len(out_features)
# out_num_features

In [124]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )
multi_window

Total window size: 240
Input indices: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167]
Label indices: [168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 

In [125]:
def compile_and_fit(model, window, patience=1000):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])
#   model.compile(loss=tf.losses.MeanSquaredError(),
#                 optimizer=tf.optimizers.Adam(),
#                 metrics=[tf.metrics.MeanSquaredError()])
  #model.compile(loss=GAIN.RMSE_loss)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [126]:
def plot_history(history):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['mean_absolute_error'], label='mae')
#     ax.plot(history.history['mean_squared_error'], label='mse')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.plot(history.history['val_mean_absolute_error'], label='val_mae')
#     ax.plot(history.history['val_mean_squared_error'], label='val_mse')
    #plt.legend(history.history.keys(), loc='upper right')
    #ax.legend(loc='upper center')
    ax.legend()
    ax.set_xlabel("epochs")
    ax.set_ylabel("loss")
    plt.show()

In [127]:
multi_window = WindowGenerator(input_width=24*7,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                               train_df=train_df,
                               val_df=val_df,
                               test_df=test_df,
                               )

**Elman_RNN**

In [128]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [129]:
elman_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
#     tf.keras.layers.SimpleRNN(128, return_sequences=True),
    tf.keras.layers.SimpleRNN(128, return_sequences=False),

    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),

    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [130]:
history = compile_and_fit(elman_model, multi_window)
multi_val_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['ELMAN_RNN'] = elman_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['ELMAN_RNN'])
print('test performance = ', multi_performance['ELMAN_RNN'])

Epoch 1/150
1/1 [==============================] - 1s 757ms/step - loss: 1.7805 - mean_absolute_error: 0.9340 - val_loss: 0.3918 - val_mean_absolute_error: 0.5492
Epoch 2/150
1/1 [==============================] - 0s 302ms/step - loss: 1.5432 - mean_absolute_error: 0.8758 - val_loss: 0.3575 - val_mean_absolute_error: 0.5143
Epoch 3/150
1/1 [==============================] - 0s 319ms/step - loss: 1.1884 - mean_absolute_error: 0.7783 - val_loss: 0.3474 - val_mean_absolute_error: 0.5224
Epoch 4/150
1/1 [==============================] - 0s 243ms/step - loss: 1.2851 - mean_absolute_error: 0.7839 - val_loss: 0.2916 - val_mean_absolute_error: 0.4594
Epoch 5/150
1/1 [==============================] - 0s 295ms/step - loss: 1.2109 - mean_absolute_error: 0.7786 - val_loss: 0.3399 - val_mean_absolute_error: 0.4881
Epoch 6/150
1/1 [==============================] - 0s 261ms/step - loss: 1.1256 - mean_absolute_error: 0.7164 - val_loss: 0.2908 - val_mean_absolute_error: 0.4162
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 279ms/step - loss: 0.1468 - mean_absolute_error: 0.2650 - val_loss: 0.0997 - val_mean_absolute_error: 0.2296
Epoch 52/150
1/1 [==============================] - 0s 306ms/step - loss: 0.1506 - mean_absolute_error: 0.2607 - val_loss: 0.1176 - val_mean_absolute_error: 0.2316
Epoch 53/150
1/1 [==============================] - 0s 296ms/step - loss: 0.1910 - mean_absolute_error: 0.2886 - val_loss: 0.1091 - val_mean_absolute_error: 0.2309
Epoch 54/150
1/1 [==============================] - 0s 249ms/step - loss: 0.1688 - mean_absolute_error: 0.2485 - val_loss: 0.1197 - val_mean_absolute_error: 0.2537
Epoch 55/150
1/1 [==============================] - 0s 262ms/step - loss: 0.1514 - mean_absolute_error: 0.2384 - val_loss: 0.0976 - val_mean_absolute_error: 0.2449
Epoch 56/150
1/1 [==============================] - 0s 265ms/step - loss: 0.1215 - mean_absolute_error: 0.2524 - val_loss: 0.1176 - val_mean_absolute_error: 0.2513
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 277ms/step - loss: 0.1023 - mean_absolute_error: 0.2229 - val_loss: 0.1313 - val_mean_absolute_error: 0.2738
Epoch 102/150
1/1 [==============================] - 0s 297ms/step - loss: 0.1430 - mean_absolute_error: 0.2674 - val_loss: 0.0982 - val_mean_absolute_error: 0.2442
Epoch 103/150
1/1 [==============================] - 0s 261ms/step - loss: 0.1260 - mean_absolute_error: 0.2382 - val_loss: 0.1171 - val_mean_absolute_error: 0.2574
Epoch 104/150
1/1 [==============================] - 0s 263ms/step - loss: 0.1014 - mean_absolute_error: 0.2261 - val_loss: 0.1103 - val_mean_absolute_error: 0.2619
Epoch 105/150
1/1 [==============================] - 0s 266ms/step - loss: 0.2051 - mean_absolute_error: 0.2496 - val_loss: 0.0925 - val_mean_absolute_error: 0.2458
Epoch 106/150
1/1 [==============================] - 0s 322ms/step - loss: 0.0989 - mean_absolute_error: 0.2129 - val_loss: 0.1336 - val_mean_absolute_error: 0.2618
Epoch 107/150
1/1 [=====

100/100 [==============================] - 9s 88ms/step - loss: 0.1549 - mean_absolute_error: 0.2577
val performance = [0.13560208678245544, 0.2881110906600952]
test performance =  [0.15486551821231842, 0.2576774060726166]


In [131]:
plot_history(history)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
multi_window.plot2(elman_model, plot_col=out_features[0])

/home/cjinw/work/tf-2/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

**GRU**

In [133]:
gru_model = tf.keras.Sequential([
    # Take the last time step.
    # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
#     tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.GRU(128, return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [134]:
history = compile_and_fit(gru_model, multi_window)
multi_val_performance['GRU'] = gru_model.evaluate(multi_window.val.repeat(-1), steps=100)
multi_performance['GRU'] = gru_model.evaluate(multi_window.test.repeat(-1), verbose=1, steps=100)
print('val performance =', multi_val_performance['GRU'])
print('test performance = ', multi_performance['GRU'])

Epoch 1/150
1/1 [==============================] - 1s 1s/step - loss: 1.8346 - mean_absolute_error: 0.9039 - val_loss: 0.3974 - val_mean_absolute_error: 0.5421
Epoch 2/150
1/1 [==============================] - 0s 414ms/step - loss: 1.7179 - mean_absolute_error: 0.9376 - val_loss: 0.3639 - val_mean_absolute_error: 0.5271
Epoch 3/150
1/1 [==============================] - 0s 377ms/step - loss: 1.6137 - mean_absolute_error: 0.8394 - val_loss: 0.3656 - val_mean_absolute_error: 0.5312
Epoch 4/150
1/1 [==============================] - 0s 361ms/step - loss: 1.1480 - mean_absolute_error: 0.7465 - val_loss: 0.3432 - val_mean_absolute_error: 0.4991
Epoch 5/150
1/1 [==============================] - 0s 380ms/step - loss: 1.5473 - mean_absolute_error: 0.8392 - val_loss: 0.3053 - val_mean_absolute_error: 0.4829
Epoch 6/150
1/1 [==============================] - 0s 407ms/step - loss: 0.9923 - mean_absolute_error: 0.7005 - val_loss: 0.2740 - val_mean_absolute_error: 0.4538
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 346ms/step - loss: 0.1549 - mean_absolute_error: 0.2479 - val_loss: 0.0646 - val_mean_absolute_error: 0.1916
Epoch 52/150
1/1 [==============================] - 0s 378ms/step - loss: 0.1266 - mean_absolute_error: 0.2420 - val_loss: 0.0790 - val_mean_absolute_error: 0.1992
Epoch 53/150
1/1 [==============================] - 0s 411ms/step - loss: 0.1584 - mean_absolute_error: 0.2751 - val_loss: 0.0631 - val_mean_absolute_error: 0.1852
Epoch 54/150
1/1 [==============================] - 0s 386ms/step - loss: 0.1946 - mean_absolute_error: 0.2761 - val_loss: 0.0690 - val_mean_absolute_error: 0.1811
Epoch 55/150
1/1 [==============================] - 0s 385ms/step - loss: 0.1213 - mean_absolute_error: 0.2285 - val_loss: 0.0584 - val_mean_absolute_error: 0.1849
Epoch 56/150
1/1 [==============================] - 0s 366ms/step - loss: 0.1164 - mean_absolute_error: 0.2169 - val_loss: 0.0814 - val_mean_absolute_error: 0.2034
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 358ms/step - loss: 0.1412 - mean_absolute_error: 0.2520 - val_loss: 0.0855 - val_mean_absolute_error: 0.2108
Epoch 102/150
1/1 [==============================] - 0s 359ms/step - loss: 0.1952 - mean_absolute_error: 0.2385 - val_loss: 0.0945 - val_mean_absolute_error: 0.2230
Epoch 103/150
1/1 [==============================] - 0s 374ms/step - loss: 0.1065 - mean_absolute_error: 0.2097 - val_loss: 0.0849 - val_mean_absolute_error: 0.2142
Epoch 104/150
1/1 [==============================] - 0s 378ms/step - loss: 0.1309 - mean_absolute_error: 0.2303 - val_loss: 0.0801 - val_mean_absolute_error: 0.2088
Epoch 105/150
1/1 [==============================] - 0s 372ms/step - loss: 0.1283 - mean_absolute_error: 0.2407 - val_loss: 0.0930 - val_mean_absolute_error: 0.2257
Epoch 106/150
1/1 [==============================] - 0s 387ms/step - loss: 0.1528 - mean_absolute_error: 0.2382 - val_loss: 0.1173 - val_mean_absolute_error: 0.2338
Epoch 107/150
1/1 [=====

100/100 [==============================] - 11s 106ms/step - loss: 0.1203 - mean_absolute_error: 0.2184
val performance = [0.07531134784221649, 0.19901034235954285]
test performance =  [0.12027297168970108, 0.2183573842048645]


**LSTM**

In [135]:
multi_lstm_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units]
    # Adding more `lstm_units` just overfits more quickly.
    #tf.keras.layers.LSTM(32, return_sequences=False),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
#     tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.LSTM(128, return_sequences=False),
    # Shape => [batch, out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [136]:
history = compile_and_fit(multi_lstm_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.val)
multi_performance['LSTM'] = multi_lstm_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['LSTM'])
print('test performance = ', multi_performance['LSTM'])

Epoch 1/150
1/1 [==============================] - 2s 2s/step - loss: 0.9880 - mean_absolute_error: 0.7224 - val_loss: 0.3182 - val_mean_absolute_error: 0.4831
Epoch 2/150
1/1 [==============================] - 0s 240ms/step - loss: 1.5375 - mean_absolute_error: 0.8776 - val_loss: 0.3597 - val_mean_absolute_error: 0.5219
Epoch 3/150
1/1 [==============================] - 0s 234ms/step - loss: 1.4115 - mean_absolute_error: 0.8290 - val_loss: 0.3934 - val_mean_absolute_error: 0.5522
Epoch 4/150
1/1 [==============================] - 0s 240ms/step - loss: 0.8981 - mean_absolute_error: 0.6826 - val_loss: 0.3495 - val_mean_absolute_error: 0.5111
Epoch 5/150
1/1 [==============================] - 0s 250ms/step - loss: 1.5962 - mean_absolute_error: 0.8164 - val_loss: 0.3780 - val_mean_absolute_error: 0.5227
Epoch 6/150
1/1 [==============================] - 0s 229ms/step - loss: 1.4530 - mean_absolute_error: 0.7853 - val_loss: 0.2278 - val_mean_absolute_error: 0.4028
Epoch 7/150
1/1 [========

1/1 [==============================] - 0s 257ms/step - loss: 0.1266 - mean_absolute_error: 0.2511 - val_loss: 0.1212 - val_mean_absolute_error: 0.2809
Epoch 52/150
1/1 [==============================] - 0s 278ms/step - loss: 0.1350 - mean_absolute_error: 0.2378 - val_loss: 0.1460 - val_mean_absolute_error: 0.2973
Epoch 53/150
1/1 [==============================] - 0s 263ms/step - loss: 0.1148 - mean_absolute_error: 0.2256 - val_loss: 0.1449 - val_mean_absolute_error: 0.2973
Epoch 54/150
1/1 [==============================] - 0s 267ms/step - loss: 0.0953 - mean_absolute_error: 0.2193 - val_loss: 0.1116 - val_mean_absolute_error: 0.2628
Epoch 55/150
1/1 [==============================] - 0s 273ms/step - loss: 0.1413 - mean_absolute_error: 0.2545 - val_loss: 0.1254 - val_mean_absolute_error: 0.2697
Epoch 56/150
1/1 [==============================] - 0s 292ms/step - loss: 0.1379 - mean_absolute_error: 0.2495 - val_loss: 0.1073 - val_mean_absolute_error: 0.2589
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 257ms/step - loss: 0.1513 - mean_absolute_error: 0.2142 - val_loss: 0.1071 - val_mean_absolute_error: 0.2714
Epoch 102/150
1/1 [==============================] - 0s 230ms/step - loss: 0.2084 - mean_absolute_error: 0.2154 - val_loss: 0.1473 - val_mean_absolute_error: 0.2957
Epoch 103/150
1/1 [==============================] - 0s 259ms/step - loss: 0.2134 - mean_absolute_error: 0.2202 - val_loss: 0.1196 - val_mean_absolute_error: 0.2780
Epoch 104/150
1/1 [==============================] - 0s 243ms/step - loss: 0.4295 - mean_absolute_error: 0.2669 - val_loss: 0.1237 - val_mean_absolute_error: 0.2773
Epoch 105/150
1/1 [==============================] - 0s 257ms/step - loss: 0.2113 - mean_absolute_error: 0.2247 - val_loss: 0.1199 - val_mean_absolute_error: 0.2751
Epoch 106/150
1/1 [==============================] - 0s 266ms/step - loss: 0.0872 - mean_absolute_error: 0.2038 - val_loss: 0.1180 - val_mean_absolute_error: 0.2679
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 119ms/step - loss: 0.1367 - mean_absolute_error: 0.3065
val performance = [0.13665708899497986, 0.3064553737640381]
test performance =  [0.10555759072303772, 0.24117766320705414]


**CNN model**

In [137]:
CONV_WIDTH = 7
CONV_LAYER_NO = 1
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -(CONV_WIDTH*CONV_LAYER_NO-CONV_LAYER_NO+1):, :]),
] + [
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(1024, activation='relu', kernel_size=(CONV_WIDTH)) for i in range(CONV_LAYER_NO)
] + [
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Dense(OUT_STEPS*out_num_features,
                          kernel_initializer=tf.initializers.zeros),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, out_num_features])
])

In [138]:
history = compile_and_fit(multi_conv_model, multi_window)

#IPython.display.clear_output()

multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

Epoch 1/150
1/1 [==============================] - 0s 479ms/step - loss: 1.2552 - mean_absolute_error: 0.7980 - val_loss: 0.3180 - val_mean_absolute_error: 0.4914
Epoch 2/150
1/1 [==============================] - 0s 215ms/step - loss: 1.6909 - mean_absolute_error: 0.8326 - val_loss: 0.3330 - val_mean_absolute_error: 0.4696
Epoch 3/150
1/1 [==============================] - 0s 223ms/step - loss: 0.8652 - mean_absolute_error: 0.6462 - val_loss: 0.2475 - val_mean_absolute_error: 0.3837
Epoch 4/150
1/1 [==============================] - 0s 260ms/step - loss: 0.9332 - mean_absolute_error: 0.7134 - val_loss: 0.2427 - val_mean_absolute_error: 0.3730
Epoch 5/150
1/1 [==============================] - 0s 203ms/step - loss: 0.4952 - mean_absolute_error: 0.4893 - val_loss: 0.2739 - val_mean_absolute_error: 0.4261
Epoch 6/150
1/1 [==============================] - 0s 245ms/step - loss: 0.6260 - mean_absolute_error: 0.5690 - val_loss: 0.3081 - val_mean_absolute_error: 0.4791
Epoch 7/150
1/1 [=====

1/1 [==============================] - 0s 223ms/step - loss: 0.1120 - mean_absolute_error: 0.2290 - val_loss: 0.1006 - val_mean_absolute_error: 0.2425
Epoch 52/150
1/1 [==============================] - 0s 220ms/step - loss: 0.1245 - mean_absolute_error: 0.2394 - val_loss: 0.1086 - val_mean_absolute_error: 0.2458
Epoch 53/150
1/1 [==============================] - 0s 221ms/step - loss: 0.1128 - mean_absolute_error: 0.2326 - val_loss: 0.1561 - val_mean_absolute_error: 0.3069
Epoch 54/150
1/1 [==============================] - 0s 260ms/step - loss: 0.2334 - mean_absolute_error: 0.2625 - val_loss: 0.1283 - val_mean_absolute_error: 0.2771
Epoch 55/150
1/1 [==============================] - 0s 242ms/step - loss: 0.1182 - mean_absolute_error: 0.2345 - val_loss: 0.1233 - val_mean_absolute_error: 0.2738
Epoch 56/150
1/1 [==============================] - 0s 223ms/step - loss: 0.1378 - mean_absolute_error: 0.2371 - val_loss: 0.1340 - val_mean_absolute_error: 0.2805
Epoch 57/150
1/1 [===========

1/1 [==============================] - 0s 241ms/step - loss: 0.1000 - mean_absolute_error: 0.2230 - val_loss: 0.0936 - val_mean_absolute_error: 0.2352
Epoch 102/150
1/1 [==============================] - 0s 275ms/step - loss: 0.0910 - mean_absolute_error: 0.2065 - val_loss: 0.1153 - val_mean_absolute_error: 0.2554
Epoch 103/150
1/1 [==============================] - 0s 216ms/step - loss: 0.3057 - mean_absolute_error: 0.2678 - val_loss: 0.1172 - val_mean_absolute_error: 0.2569
Epoch 104/150
1/1 [==============================] - 0s 223ms/step - loss: 0.0886 - mean_absolute_error: 0.2094 - val_loss: 0.0721 - val_mean_absolute_error: 0.2123
Epoch 105/150
1/1 [==============================] - 0s 251ms/step - loss: 0.2269 - mean_absolute_error: 0.2189 - val_loss: 0.1200 - val_mean_absolute_error: 0.2560
Epoch 106/150
1/1 [==============================] - 0s 243ms/step - loss: 0.1093 - mean_absolute_error: 0.2254 - val_loss: 0.0880 - val_mean_absolute_error: 0.2327
Epoch 107/150
1/1 [=====

1/1 [==============================] - 0s 107ms/step - loss: 0.0834 - mean_absolute_error: 0.2216
val performance = [0.08342908322811127, 0.22159512341022491]
test performance =  [0.23372162878513336, 0.250281423330307]


In [139]:
multi_val_performance['Conv'] = multi_conv_model.evaluate(multi_window.val)
multi_performance['Conv'] = multi_conv_model.evaluate(multi_window.test, verbose=0)
print('val performance =', multi_val_performance['Conv'])
print('test performance = ', multi_performance['Conv'])

1/1 [==============================] - 0s 148ms/step - loss: 0.1018 - mean_absolute_error: 0.2398
val performance = [0.10181491076946259, 0.23976628482341766]
test performance =  [0.11338125169277191, 0.2315313071012497]


**performance**

In [140]:
x = np.arange(len(multi_performance))
width = 0.3


metric_name = 'mean_absolute_error'
metric_index = multi_conv_model.metrics_names.index('mean_absolute_error')
val_mae = [v[metric_index] for v in multi_val_performance.values()]
test_mae = [v[metric_index] for v in multi_performance.values()]

plt.figure()
plt.bar(x - 0.17, val_mae, width, label='Validation')
plt.bar(x + 0.17, test_mae, width, label='Test')
plt.xticks(ticks=x, labels=multi_performance.keys(),
           rotation=45)
plt.ylabel(f'MAE (average over all times and outputs)')
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [141]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 총질소
mae:
0.23453677
mse:
0.10882394
rmse
0.32988474
mape
120.15389204025269


In [142]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총질소
mae:
0.23453677
mse:
0.10882394
rmse
0.32988474
mape
120.15389204025269


In [143]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

column : 총질소
mae:
0.06616222
mse:
0.00641157
rmse
0.080072284
mape
25.37636160850525


In [144]:
multi_window.compa(elman_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=True)

column : 총질소
mae:
0.06616222
mse:
0.00641157
rmse
0.080072284
mape
25.37636160850525


In [145]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 총질소
mae:
0.2169208
mse:
0.106169716
rmse
0.32583696
mape
94.7087049484253


In [146]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총질소
mae:
0.2169208
mse:
0.106169716
rmse
0.32583696
mape
94.7087049484253


In [147]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

column : 총질소
mae:
0.08662766
mse:
0.009508272
rmse
0.09751037
mape
32.526981830596924


In [148]:
multi_window.compa(gru_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=True)

column : 총질소
mae:
0.08662766
mse:
0.009508272
rmse
0.09751037
mape
32.526981830596924


In [149]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 총질소
mae:
0.16950367
mse:
0.065992564
rmse
0.25689018
mape
78.33333015441895


In [150]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총질소
mae:
0.16950367
mse:
0.065992564
rmse
0.25689018
mape
78.33333015441895


In [151]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

column : 총질소
mae:
0.047800377
mse:
0.0034304957
rmse
0.058570433
mape
18.1416854262352


In [152]:
multi_window.compa(multi_lstm_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=True)

column : 총질소
mae:
0.047800377
mse:
0.0034304957
rmse
0.058570433
mape
18.1416854262352


In [153]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=False)

column : 총질소
mae:
0.19188605
mse:
0.07365972
rmse
0.27140325
mape
84.64800715446472


In [154]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=False)

column : 총질소
mae:
0.19188605
mse:
0.07365972
rmse
0.27140325
mape
84.64800715446472


In [155]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example2, min_max_normailze=True)

column : 총질소
mae:
0.099879764
mse:
0.011118242
rmse
0.105443075
mape
34.75733399391174


In [156]:
multi_window.compa(multi_conv_model, plot_col=out_features[0], windows=multi_window.example3, min_max_normailze=True)

column : 총질소
mae:
0.099879764
mse:
0.011118242
rmse
0.105443075
mape
34.75733399391174
